# Using Python for Research Homework: Week 4, Case Study 3

Homophily is a property of networks.  Homophily occurs when nodes that are neighbors in a network also share a characteristic more often than nodes that are not network neighbors.  In this case study, we will investigate homophily of several characteristics of individuals connected in social networks in rural India.

### Exercise 1
In Exercise 1, we will calculate the chance homophily for an arbitrary characteristic. Homophily is the proportion of edges in the network whose constituent nodes share that characteristic.

How much homophily do we expect by chance? If characteristics are distributed completely randomly, the probability that two nodes \(x\) and \(y\) share characteristic \(a\) is the probability both nodes have characteristic \(a\) , which is the marginal probability of \(a\) squared. The total probability that nodes \(x\) and \(y\) share their characteristic is therefore the sum of the square of the marginal probabilities of each characteristic in the network.

<strong>Instructions</strong>
<ul><li>Create a function <code>marginal_prob</code> that takes a dictionary <code>chars</code> with personal IDs as keys and characteristics as values; it should return a dictionary with characteristics as keys and their marginal probability (frequency of occurence of a characteristic divided by the sum of frequencies of each characteristic) as values.</li>
<li>Create a function <code>chance_homophily(chars)</code> that takes a dictionary <code>chars</code> defined as above and computes the chance homophily (homophily due to chance alone) for that characteristic.</li>
<li>A sample of three peoples' favorite colors is given in <code>favorite_colors</code>. Use your function to compute the chance homophily in this group, and store it as <code>color_homophily</code>.</li>
<li>Print <code>color_homophily</code>.</li></ul>

In [2]:
from collections import Counter
import numpy as np

def marginal_prob(chars):
    counts = Counter(chars.values())
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}

def chance_homophily(chars):
    probs = marginal_prob(chars)
    return sum(p * p for p in probs.values())

favorite_colors = {
    "ankit":  "red",
    "xiaoyu": "blue",
    "mary":   "blue"
}

color_homophily = chance_homophily(favorite_colors)
print(color_homophily)

0.5555555555555556


### Exercise 2

In the remaining exercises, we will calculate actual homophily in these village and compare the obtained values to those obtained by chance. In this exercise, we subset the data into individual villages and store them.

#### Instructions 

- `individual_characteristics.dta` contains several characteristics for each individual in the dataset such as age, religion, and caste. Use the `pandas` library to read in and store these characteristics as a dataframe called `df`.
- Store separate datasets for individuals belonging to Villages 1 and 2 as `df1` and `df2`, respectively.
- Note that some attributes may be missing for some individuals. In this case study, we will ignore rows of data where some column information is missing.
- Use the head method to display the first few entries of `df1`.

In [3]:
import pandas as pd

df  = pd.read_csv("C:/Users/User/OneDrive/Desktop/python_for_research/data/individual_characteristics.csv", low_memory=False, index_col=0)

df1 = df[df["village"] == 1]
df2 = df[df["village"] == 2]

print(df1.head())

   village  adjmatrix_key     pid  hhid  resp_id  resp_gend  \
0        1              5  100201  1002        1          1   
1        1              6  100202  1002        2          2   
2        1             23  100601  1006        1          1   
3        1             24  100602  1006        2          2   
4        1             27  100701  1007        1          1   

                   resp_status  age  religion caste  ...       privategovt  \
0            Head of Household   38  HINDUISM   OBC  ...  PRIVATE BUSINESS   
1  Spouse of Head of Household   27  HINDUISM   OBC  ...               NaN   
2            Head of Household   29  HINDUISM   OBC  ...        OTHER LAND   
3  Spouse of Head of Household   24  HINDUISM   OBC  ...  PRIVATE BUSINESS   
4            Head of Household   58  HINDUISM   OBC  ...        OTHER LAND   

  work_outside work_outside_freq shgparticipate shg_no savings savings_no  \
0          Yes                 0             No    NaN      No        NaN  

### Exercise 3 

In this exercise, we define a few dictionaries that enable us to look up the sex, caste, and religion of members of each village by personal ID. For Villages 1 and 2, their personal IDs are stored as `pid`.

#### Instructions 
- Define dictionaries with personal IDs as keys and a given covariate for that individual as values. Complete this for the sex, caste, and religion covariates, for Villages 1 and 2.
- For Village 1, store these dictionaries into variables named `sex1`, `caste1`, and `religion1`.
- For Village 2, store these dictionaries into variables named `sex2`, `caste2`, and `religion2`.

In [5]:
def make_dict(df, col):
    """Create a dictionary keyed by the PID column with values from `col`."""
    tmp = df.dropna(subset=['pid', col]).copy()
    return tmp.set_index('pid')[col].to_dict()

sex1 = make_dict(df1, 'resp_gend')
caste1 = make_dict(df1, 'caste')
religion1 = make_dict(df1, 'religion')

sex2 = make_dict(df2, 'resp_gend')
caste2 = make_dict(df2, 'caste')
religion2 = make_dict(df2, 'religion')

### Exercise 4

In this exercise, we will print the chance homophily of several characteristics of Villages 1 and 2. 

#### Instructions 
-  Use `chance_homophily` to compute the chance homophily for sex, caste, and religion In Villages 1 and 2. Is the chance homophily for any attribute very high for either village?

In [6]:
print("Chance of homophily for village 1's sex", chance_homophily(sex1))
print("Chance of homophily for village 1's caste", chance_homophily(caste1))
print("Chance of homophily for village 1's religion", chance_homophily(religion1))

print("Chance of homophily for village 2's sex", chance_homophily(sex2))
print("Chance of homophily for village 2's caste", chance_homophily(caste2))
print("Chance of homophily for village 2's religion", chance_homophily(religion2))

# For village 2, the chance homophily of religion is 1 meaning that everyone has the same religion.

Chance of homophily for village 1's sex 0.5027299861680701
Chance of homophily for village 1's caste 0.6741488509791551
Chance of homophily for village 1's religion 0.9804896988521925
Chance of homophily for village 2's sex 0.5005945303210464
Chance of homophily for village 2's caste 0.425368244800893
Chance of homophily for village 2's religion 1.0


### Exercise 5

In this exercise, we will create a function that computes the observed homophily given a village and characteristic.

#### Instructions 
- Complete the function `homophily()`, which takes a network `G`, a dictionary of node characteristics `chars`, and node IDs `IDs`. For each node pair, determine whether a tie exists between them, as well as whether they share a characteristic. The total count of these is `num_ties` and `num_same_ties`, respectively, and their ratio is the homophily of chars in `G`. Complete the function by choosing where to increment `num_same_ties` and `num_ties`.

In [7]:
def homophily(G, chars, IDs):
    """
    Given a network G, a dict of characteristics chars for node IDs,
    and dict of node IDs for each node in the network,
    find the homophily of the network.
    """
    num_same_ties = 0
    num_ties = 0
    for n1, n2 in G.edges():
        if IDs[n1] in chars and IDs[n2] in chars:
            if G.has_edge(n1, n2):
                num_ties += 1
                if chars[IDs[n1]] == chars[IDs[n2]]:
                    num_same_ties +=1
    return (num_same_ties / num_ties)  

### Exercise 6

In this exercise, we will obtain the personal IDs for Villages 1 and 2. These will be used in the next exercise to calculate homophily for these villages.

#### Instructions 
- In this dataset, each individual has a personal ID, or PID, stored in `key_vilno_1.csv` and `key_vilno_2.csv` for villages 1 and 2, respectively. `data_filepath1` and `data_filepath2` contain the URLs to the datasets used in this exercise. Use `pd.read_csv` to read in and store `key_vilno_1.csv` and `key_vilno_2.csv` as `pid1` and `pid2` respectively. 

In [8]:
data_filepath1 = "C:/Users/User/OneDrive/Desktop/python_for_research/data/key_vilno_1.csv"
data_filepath2 = "C:/Users/User/OneDrive/Desktop/python_for_research/data/key_vilno_2.csv"

pid1 = pd.read_csv(data_filepath1)
pid2 = pd.read_csv(data_filepath2)


### Exercise 7

In this exercise, we will compute the homophily of several network characteristics for Villages 1 and 2 and compare them to homophily due to chance alone. The networks for these villages have been stored as networkx graph objects `G1` and `G2`.

#### Instructions 

- Use your `homophily()` function to compute the observed homophily for sex, caste, and religion in Villages 1 and 2. Print all six values.
- Use the `chance_homophily()` to compare these values to chance homophily. Are these values higher or lower than that expected by chance?

In [10]:
import networkx as nx
A1 = np.array(pd.read_csv("C:/Users/User/OneDrive/Desktop/python_for_research/data/adj_allVillageRelationships_vilno_1.csv", index_col=0))
A2 = np.array(pd.read_csv("C:/Users/User/OneDrive/Desktop/python_for_research/data/adj_allVillageRelationships_vilno_2.csv", index_col=0))
G1 = nx.to_networkx_graph(A1)
G2 = nx.to_networkx_graph(A2)

observed = {} # Compute observed homophily for sex, caste, religion in both villages
chance_vals = {} # Compute chance homophily for sex, caste, religion in both villages

pid1 = pd.read_csv(data_filepath1, dtype=int)['0'].to_dict()
pid2 = pd.read_csv(data_filepath2, dtype=int)['0'].to_dict()

observed['v1_sex'] = homophily(G1, sex1, pid1)
observed['v1_caste'] = homophily(G1, caste1, pid1)
observed['v1_religion'] = homophily(G1, religion1, pid1)
chance_vals['v1_sex'] = chance_homophily(sex1)
chance_vals['v1_caste'] = chance_homophily(caste1)
chance_vals['v1_religion'] = chance_homophily(religion1)

observed['v2_sex'] = homophily(G2, sex2, pid2)
observed['v2_caste'] = homophily(G2, caste2, pid2)
observed['v2_religion'] = homophily(G2, religion2, pid2)
chance_vals['v2_sex'] = chance_homophily(sex2)
chance_vals['v2_caste'] = chance_homophily(caste2)
chance_vals['v2_religion'] = chance_homophily(religion2)

print(f"Village 1 sex: observed homophily = {observed['v1_sex']:.4f}, chance homophily = {chance_vals['v1_sex']:.4f}")
print(f"Village 1 caste: observed homophily = {observed['v1_caste']:.4f}, chance homophily = {chance_vals['v1_caste']:.4f}")
print(f"Village 1 religion: observed homophily = {observed['v1_religion']:.4f}, chance homophily = {chance_vals['v1_religion']:.4f}")
print(f"Village 2 sex: observed homophily = {observed['v2_sex']:.4f}, chance homophily = {chance_vals['v2_sex']:.4f}")
print(f"Village 2 caste: observed homophily = {observed['v2_caste']:.4f}, chance homophily = {chance_vals['v2_caste']:.4f}")
print(f"Village 2 religion: observed homophily = {observed['v2_religion']:.4f}, chance homophily = {chance_vals['v2_religion']:.4f}")


Village 1 sex: observed homophily = 0.5138, chance homophily = 0.5027
Village 1 caste: observed homophily = 0.8226, chance homophily = 0.6741
Village 1 religion: observed homophily = 0.9908, chance homophily = 0.9805
Village 2 sex: observed homophily = 0.4962, chance homophily = 0.5006
Village 2 caste: observed homophily = 0.8564, chance homophily = 0.4254
Village 2 religion: observed homophily = 1.0000, chance homophily = 1.0000


### Interpretation of Homophily Results

**Village 1 – Sex:**  
Observed (0.5138) is slightly higher than chance (0.5027).  
→ People tend to connect a bit more with others of the same sex than expected by random chance.

**Village 1 – Caste:**  
Observed (0.8226) is much higher than chance (0.6741).  
→ Strong caste-based homophily; people mostly connect within their caste.

**Village 1 – Religion:**  
Observed (0.9908) is slightly higher than chance (0.9805).  
→ Almost everyone shares the same religion, so homophily is near total.

**Village 2 – Sex:**  
Observed (0.4962) is slightly lower than chance (0.5006).  
→ Connections are roughly random by sex.

**Village 2 – Caste:**  
Observed (0.8564) is much higher than chance (0.4254).  
→ Very strong caste-based homophily, stronger than in Village 1.

**Village 2 – Religion:**  
Observed and chance are both 1.0000.  
→ All individuals share the same religion, so complete homogeneity.

**Summary:**  
Both villages show strong **caste-based homophily**, near-total **religious homogeneity**,  
and almost **random mixing by sex**.